In [1]:
import pandas as pd
import inro.modeller as m

mm = m.Modeller()
eb = mm.emmebank
sc_1 = eb.scenario(6002)
sc_2 = eb.scenario(1000)
ntwk1 = sc_1.get_network()
ntwk2 = sc_2.get_network()

In [2]:
ol_nodes = []
for link in ntwk1.links():
    if link["@ol_link"] == 1:
        if link.i_node not in ol_nodes:
            ol_nodes.append(link.i_node)
        if link.j_node not in ol_nodes:
            ol_nodes.append(link.j_node)

In [3]:
for node in ol_nodes:
    node2 = ntwk2.node(node.id)
    node.x = node2.x
    node.y = node2.y

In [4]:
sc_1.publish_network(ntwk1)

In [5]:
for link in ntwk1.links():
    if link["@ol_link"] == 1:
        olLink = ntwk2.link(link.i_node.id,link.j_node.id)
        newVerts = olLink.vertices
        link.vertices = newVerts
        link.length = olLink.length

In [6]:
sc_1.publish_network(ntwk1)

In [7]:
linkatts_1 = ntwk1.attributes("LINK")

In [8]:
linkatts_2 = ntwk2.attributes("LINK")

In [12]:
linkatts_2.remove(u'additional_volume')
linkatts_2.remove(u'auto_time')
linkatts_2.remove(u'auto_volume')
linkatts_2.remove(u'aux_transit_volume')

In [14]:
for zone in ntwk1.centroids():
    if zone["@ol_zone"] == 1:
        
        zone2 = ntwk2.node(zone.id)
        zone.x = zone2.x
        zone.y = zone2.y
        
        for link in zone.incoming_links():
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
        for link in zone.outgoing_links():
            ntwk1.delete_link(link.i_node.id,link.j_node.id)    
        
        for link in zone2.incoming_links():
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]        
        for link in zone2.outgoing_links():
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]

In [15]:
sc_1.publish_network(ntwk1)

In [19]:
ntwk1.link(10611,97085).modes == set([ntwk1.mode('t')])

True

In [21]:
for node in ol_nodes:
    for link in node.incoming_links():
        if link.modes == set([ntwk1.mode('t')]):
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
    for link in node.outgoing_links():
        if link.modes == set([ntwk1.mode('t')]):
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
    newNode = ntwk2.node(node.id)
    for link in newNode.incoming_links():
        if link.modes == set([ntwk2.mode('t')]):
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att] 
    for link in newNode.outgoing_links():
        if link.modes == set([ntwk2.mode('t')]):
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]

In [22]:
sc_1.publish_network(ntwk1)